In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

2023-11-01 13:45:40.355665: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-01 13:45:40.375008: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-01 13:45:40.735064: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [27]:
df = pd.read_csv('../../Data/data_lake_raw/RDC_Inventory_Core_Metrics_Zip_History.csv').iloc[:-1]


/tmp/ipykernel_19818/3505859450.py:1: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../../Data/data_lake_raw/RDC_Inventory_Core_Metrics_Zip_History.csv').iloc[:-1]


In [28]:
df.shape

(2465722, 40)

In [29]:
# get max/min dates for i rates

df['month_date_yyyymm'] = df['month_date_yyyymm'].astype(int)

print(df['month_date_yyyymm'].min())

print(df['month_date_yyyymm'].max())

201607
202309


In [30]:
cols_to_drop = df.filter(regex='_yy$|_mm$', axis=1).columns

df = df.drop(cols_to_drop, axis=1)

In [31]:
df.head(3)

,month_date_yyyymm,postal_code,zip_name,median_listing_price,active_listing_count,median_days_on_market,new_listing_count,price_increased_count,price_reduced_count,pending_listing_count,median_listing_price_per_square_foot,median_square_feet,average_listing_price,total_listing_count,pending_ratio,quality_flag
0,202309,55127,"saint paul, mn",635000.0,53.0,39.0,36.0,0.0,20.0,50.0,235.0,2708.0,812323.0,106.0,0.9434,0.0
1,202309,18354,"sciota, pa",399999.0,2.0,195.0,0.0,0.0,0.0,2.0,113.0,3564.0,399999.0,3.0,1.0000,1.0
2,202309,35593,"vina, al",275000.0,1.0,47.0,0.0,0.0,0.0,1.0,84.0,3256.0,275000.0,2.0,1.0000,1.0


In [34]:
data_types = df.dtypes

empty_values_count = df.isnull().sum()

empty_values_percent = (empty_values_count / len(df)) * 100

result = pd.DataFrame({
    'Column Name': df.columns,
    'Data Type': data_types.values,
    'Empty Values Count': empty_values_count.values,
    'Empty Values Percent': empty_values_percent.values
})

result


,Column Name,Data Type,Empty Values Count,Empty Values Percent
0,month_date_yyyymm,int64,0,0.000000
1,postal_code,object,0,0.000000
2,zip_name,object,69791,2.830449
3,median_listing_price,float64,7016,0.284541
4,active_listing_count,float64,4730,0.191830
5,median_days_on_market,float64,25630,1.039452
6,new_listing_count,float64,3279,0.132983
7,price_increased_count,float64,3279,0.132983
8,price_reduced_count,float64,3279,0.132983
9,pending_listing_count,float64,555169,22.515474


In [49]:
# see if I can update zip_names
mapping = df.dropna(subset=['zip_name']).set_index('postal_code')['zip_name'].to_dict()
df['zip_name'] = df['postal_code'].map(mapping).where(df['zip_name'].isna(), df['zip_name'])

In [67]:
row_counts = df['postal_code'].value_counts()

row_counts.head()

postal_code
55127    87
61732    87
59047    87
94402    87
66083    87
Name: count, dtype: int64

In [44]:
len(df['postal_code'].unique().tolist())

481.7701149425287

In [43]:
len(df['month_date_yyyymm'].unique().tolist())

87